In [14]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings

pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

pd.set_option('display.max_colwidth', None)

In [15]:
#Cargamos los CSV y comprobamos
data_company_overview = pd.read_csv("1_overview.csv")
data_balance = pd.read_csv("2_balance.csv")
data_cash_flow = pd.read_csv("3_cash_flow.csv")
data_income_statement = pd.read_csv("4_income.csv")
data_market = pd.read_csv("5_stocks.csv", sep=';')
data_balance_t = pd.read_csv("2_balance_t.csv")
data_cash_flow_t = pd.read_csv("3_cash_flow_t.csv")
data_income_statement_t = pd.read_csv("4_income_t.csv")

In [16]:
overview_df = data_company_overview.copy()
balance_df = data_balance.copy()
cf_df = data_cash_flow.copy()
income_df = data_income_statement.copy()
market_df = data_market.copy()
balance_t_df = data_balance_t.copy()
cf_t_df = data_cash_flow_t.copy()
income_t_df = data_income_statement_t.copy()

In [17]:
for col in ['open', 'high', 'low', 'close']:
    market_df[col] = market_df[col].str.replace(',', '.').astype(float)

In [18]:
def date2col(df, col):
    df[col] = pd.to_datetime(df[col])

In [19]:
date2col(balance_df, 'fiscal_date_ending')
date2col(cf_df, 'fiscal_date_ending')
date2col(income_df, 'fiscal_date_ending')
date2col(market_df, 'date')
date2col(balance_t_df, 'fiscal_date_ending')
date2col(cf_t_df, 'fiscal_date_ending')
date2col(income_t_df, 'fiscal_date_ending')

In [42]:
new_market_df = market_df.copy()

# Identificar el último día del año para cada combinación de 'symbol' y 'year'
new_market_df['year'] = new_market_df['date'].dt.year
new_market_df['last_day_of_year'] = new_market_df.groupby(['symbol', 'year'])['date'].transform('max')

# Crear la nueva columna
new_market_df['end_of_year_market_cap'] = new_market_df.apply(
    lambda row: row['market_cap'] if row['date'] == row['last_day_of_year'] else None,
    axis=1
)

# Elimina la columna auxiliar si no es necesaria
new_market_df.drop(columns=['year', 'last_day_of_year'], inplace=True)

# Verifica los resultados
new_market_df.head()

# Guardar el DataFrame en un archivo CSV
new_market_df.to_csv("5.new_market2_df.csv", index=False)
new_market_df.to_csv("5.new_market2_df_comas.csv", index=False, decimal=',', sep=';')

In [ ]:
balance_df['fiscal_date_ending'] = pd.to_datetime(balance_df['fiscal_date_ending'])
balance_t_df['fiscal_date_ending'] = pd.to_datetime(balance_t_df['fiscal_date_ending'])
market_df['date'] = pd.to_datetime(market_df['date'])

# Obtener un conjunto de fechas hábiles del mercado
market_dates = market_df['date'].sort_values().unique()

# Función para ajustar las fechas al día hábil más cercano
def closest_market_date(date, market_dates):
    if date in market_dates:
        return date
    else:
        # Buscar la fecha válida más cercana
        closest_date = min(market_dates, key=lambda x: abs(x - date))
        return closest_date

# Aplicar la función al DataFrame izquierdo
balance_df['adjusted_date'] = balance_df['fiscal_date_ending'].apply(lambda x: closest_market_date(x, market_dates))
balance_t_df['adjusted_date'] = balance_t_df['fiscal_date_ending'].apply(lambda x: closest_market_date(x, market_dates))

# Hacer el merge con las fechas ajustadas
merged_df = (
    balance_df.merge(cf_df, on=["symbol", "year"], how="inner", suffixes=("", "_dup"))
    .merge(income_df, on=["symbol", "year"], how="inner", suffixes=("", "_dup"))
    .merge(market_df, left_on=["symbol", "adjusted_date"], right_on=["symbol", "date"], how="inner", suffixes=("", "_dup"))
)
merged_t_df = (
    balance_t_df.merge(cf_t_df, on=["symbol", "year", "quarter"], how="inner", suffixes=("", "_dup"))
    .merge(income_t_df, on=["symbol", "year", "quarter"], how="inner", suffixes=("", "_dup"))
    .merge(market_df, left_on=["symbol", "adjusted_date"], right_on=["symbol", "date"], how="inner", suffixes=("", "_dup"))
)

# Identificar columnas duplicadas generadas por sufijos
columns_to_drop = [col for col in merged_df.columns if col.endswith("_dup")]
columns_to_drop_t = [col for col in merged_t_df.columns if col.endswith("_dup")]

# Dropear columnas duplicadas
merged_df = merged_df.drop(columns=columns_to_drop)
merged_t_df = merged_t_df.drop(columns=columns_to_drop)


In [34]:
# Calcular ratios
merged_df["gross_margin"] = round((merged_df["gross_profit"] / merged_df["total_revenue"]) * 100,2)
merged_df["net_profit_margin"] = round((merged_df["net_income"] / merged_df["total_revenue"]) * 100,2)
merged_df["roe"] = round((merged_df["net_income"] / merged_df["total_shareholder_equity"]) * 100,2)
merged_df["roa"] = round((merged_df["net_income"] / merged_df["total_assets"]) * 100,2)
merged_df["current_ratio"] = round(merged_df["total_current_assets"] / merged_df["total_current_liabilities"],2)
merged_df["debt_to_equity"] = round(merged_df["total_liabilities"] / merged_df["total_shareholder_equity"],2)
merged_df["debt_to_assets"] = round(merged_df["total_liabilities"] / merged_df["total_assets"],2)
merged_df['eps'] = round(merged_df['net_income'] / merged_df['shares'],2)
merged_df['p/e_ratio'] = round(merged_df['close'] / merged_df['eps'],2)
merged_df['p/s_ratio'] = round(merged_df['market_cap'] / merged_df['total_revenue'],2)
merged_df['p/b_ratio'] = round(merged_df['market_cap'] / merged_df['total_shareholder_equity'],2)
merged_df['dividend_per_share'] = round(merged_df['dividend_payout_common_stock'] / merged_df['shares'],2)
merged_df['dividend_yield'] = round((merged_df['dividend_per_share'] / merged_df['close']) * 100,2)

# Crear un DataFrame con todos los ratios calculados
ratios_df = merged_df[
    ["symbol",'name','company_id','sector', "year", "gross_margin", "net_profit_margin", "roe", "roa", 
     "current_ratio", "debt_to_equity", "debt_to_assets",
     "eps", "p/e_ratio", "p/s_ratio", "p/b_ratio", "dividend_per_share", "dividend_yield"]
]

#ratios_df["end_of_year_market_cap"] = merged_df["market_cap"]

# Mostrar un resumen de los ratios
ratios_df

,symbol,name,company_id,sector,year,gross_margin,net_profit_margin,roe,roa,current_ratio,debt_to_equity,debt_to_assets,eps,p/e_ratio,p/s_ratio,p/b_ratio,dividend_per_share,dividend_yield
0,CMCSA,Comcast Corp,0,Comunicacion,2023,88.09,12.66,18.61,5.81,0.60,2.20,0.69,4.03,10.88,1.38,2.02,1.25,2.85
1,CMCSA,Comcast Corp,0,Comunicacion,2022,80.82,4.42,6.63,2.09,0.78,2.18,0.69,1.41,24.80,1.10,1.65,1.24,3.55
2,CMCSA,Comcast Corp,0,Comunicacion,2021,80.60,12.17,14.73,5.13,0.85,1.87,0.65,3.71,13.57,1.65,2.00,1.19,2.36
3,CMCSA,Comcast Corp,0,Comunicacion,2020,79.91,10.17,11.66,3.85,0.93,2.03,0.67,2.76,18.99,1.93,2.21,1.08,2.06
4,CMCSA,Comcast Corp,0,Comunicacion,2019,80.03,11.99,15.78,4.96,0.84,2.18,0.69,3.42,13.15,1.58,2.07,0.98,2.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,META,Meta Platforms Inc.,54,Tecnologia de la Informacion,2022,78.30,19.94,18.45,12.49,2.20,0.48,0.32,8.82,13.64,2.72,2.52,0.00,0.00
325,META,Meta Platforms Inc.,54,Tecnologia de la Informacion,2021,80.77,33.43,31.53,23.72,3.15,0.33,0.25,14.98,22.45,7.51,7.08,0.00,0.00
326,META,Meta Platforms Inc.,54,Tecnologia de la Informacion,2020,80.55,33.96,22.72,18.29,5.05,0.24,0.19,11.09,24.63,8.37,5.60,0.00,0.00
327,META,Meta Platforms Inc.,54,Tecnologia de la Informacion,2019,81.91,26.19,18.29,13.86,4.40,0.32,0.24,7.03,29.20,7.65,5.34,0.89,0.43


In [35]:
# Calcular ratios
merged_t_df["gross_margin"] = round((merged_t_df["gross_profit"] / merged_t_df["total_revenue"]) * 100,2)
merged_t_df["net_profit_margin"] = round((merged_t_df["net_income"] / merged_t_df["total_revenue"]) * 100,2)
merged_t_df["roe"] = round((merged_t_df["net_income"] / merged_t_df["total_shareholder_equity"]) * 100,2)
merged_t_df["roa"] = round((merged_t_df["net_income"] / merged_t_df["total_assets"]) * 100,2)
merged_t_df["current_ratio"] = round(merged_t_df["total_current_assets"] / merged_t_df["total_current_liabilities"],2)
merged_t_df["debt_to_equity"] = round(merged_t_df["total_liabilities"] / merged_t_df["total_shareholder_equity"],2)
merged_t_df["debt_to_assets"] = round(merged_t_df["total_liabilities"] / merged_t_df["total_assets"],2)
merged_t_df['eps'] = round(merged_t_df['net_income'] / merged_t_df['shares'],2)
merged_t_df['p/e_ratio'] = round(merged_t_df['close'] / merged_t_df['eps'],2)
merged_t_df['p/s_ratio'] = round(merged_t_df['market_cap'] / merged_t_df['total_revenue'],2)
merged_t_df['p/b_ratio'] = round(merged_t_df['market_cap'] / merged_t_df['total_shareholder_equity'],2)
merged_t_df['dividend_per_share'] = round(merged_t_df['dividend_payout_common_stock'] / merged_t_df['shares'],2)
merged_t_df['dividend_yield'] = round((merged_t_df['dividend_per_share'] / merged_t_df['close']) * 100,2)

# Crear un DataFrame con todos los ratios calculados
ratios_t_df = merged_t_df[
    ["symbol",'name','company_id','sector', "year", "quarter", "gross_margin", "net_profit_margin", "roe", "roa", 
     "current_ratio", "debt_to_equity", "debt_to_assets",
     "eps", "p/e_ratio", "p/s_ratio", "p/b_ratio", "dividend_per_share", "dividend_yield"]
]

#ratios_t_df["end_of_year_market_cap"] = merged_t_df["market_cap"]

# Mostrar un resumen de los ratios
ratios_t_df

,symbol,name,company_id,sector,year,quarter,gross_margin,net_profit_margin,roe,roa,current_ratio,debt_to_equity,debt_to_assets,eps,p/e_ratio,p/s_ratio,p/b_ratio,dividend_per_share,dividend_yield
0,CMCSA,Comcast Corp,0,Comunicacion,2023,Q4,88.44,10.43,3.94,1.23,0.60,2.20,0.69,0.85,51.59,5.36,2.02,0.31,0.71
1,CMCSA,Comcast Corp,0,Comunicacion,2023,Q3,88.73,13.44,4.90,1.55,0.70,2.16,0.68,1.06,41.83,5.62,2.05,0.31,0.70
2,CMCSA,Comcast Corp,0,Comunicacion,2023,Q2,87.89,13.92,5.05,1.62,0.76,2.12,0.68,1.11,37.43,5.20,1.89,0.32,0.77
3,CMCSA,Comcast Corp,0,Comunicacion,2023,Q1,87.30,12.91,4.65,1.48,0.69,2.15,0.68,1.00,37.91,4.87,1.76,0.31,0.82
4,CMCSA,Comcast Corp,0,Comunicacion,2022,Q4,87.72,9.89,3.73,1.18,0.78,2.18,0.69,0.79,44.27,4.37,1.65,0.31,0.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,META,Meta Platforms Inc.,54,Tecnologia de la Informacion,2019,Q1,81.32,16.11,2.81,2.22,5.38,0.27,0.21,0.92,183.37,29.42,5.13,0.00,0.00
1309,META,Meta Platforms Inc.,54,Tecnologia de la Informacion,2018,Q4,83.25,41.23,8.18,7.07,7.19,0.16,0.14,2.62,50.03,20.65,4.10,0.00,0.00
1310,META,Meta Platforms Inc.,54,Tecnologia de la Informacion,2018,Q3,82.39,37.42,6.39,5.56,9.00,0.15,0.13,1.95,83.30,31.11,5.32,0.00,0.00
1311,META,Meta Platforms Inc.,54,Tecnologia de la Informacion,2018,Q2,83.27,38.59,6.43,5.66,10.67,0.14,0.12,1.94,100.16,38.61,6.44,0.00,0.00


In [36]:
ratios_df.reset_index(drop=False,inplace=True)
ratios_df.rename(columns={"index": "ratios_id"}, inplace=True)
ratios_t_df.reset_index(drop=False,inplace=True)
ratios_t_df.rename(columns={"index": "ratios_t_id"}, inplace=True)

In [24]:
ratios_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ratios_id           329 non-null    int64  
 1   symbol              329 non-null    object 
 2   name                329 non-null    object 
 3   company_id          329 non-null    int64  
 4   sector              329 non-null    object 
 5   year                329 non-null    int64  
 6   gross_margin        329 non-null    float64
 7   net_profit_margin   329 non-null    float64
 8   roe                 329 non-null    float64
 9   roa                 329 non-null    float64
 10  current_ratio       329 non-null    float64
 11  debt_to_equity      329 non-null    float64
 12  debt_to_assets      329 non-null    float64
 13  eps                 329 non-null    float64
 14  p/e_ratio           329 non-null    float64
 15  p/s_ratio           329 non-null    float64
 16  p/b_rati

In [37]:
# Guardar el DataFrame de ratios en un archivo CSV
ratios_df.to_csv("6_ratios.csv", index=False)
ratios_df.to_csv("ratios_comas.csv", index=False, decimal=',', sep=';')
ratios_t_df.to_csv("6_ratios_t.csv", index=False)
ratios_t_df.to_csv("ratios_t_comas.csv", index=False, decimal=',', sep=';')

In [38]:
# Guardar el DataFrame de ratios en un archivo CSV
merged_df.to_csv("7_merged.csv", index=False)
merged_t_df.to_csv("7_t_merged.csv", index=False)

In [28]:
merged_df

,balance_id,fiscal_date_ending,total_assets,total_current_assets,cash_and_cash_equivalents_at_carrying_value,cash_and_short_term_investments,inventory,current_net_receivables,total_non_current_assets,property_plant_equipment,accumulated_depreciation_amortization_ppe,intangible_assets,intangible_assets_excluding_goodwill,goodwill,investments,long_term_investments,short_term_investments,other_current_assets,other_non_current_assets,total_liabilities,total_current_liabilities,current_accounts_payable,deferred_revenue,current_debt,short_term_debt,total_non_current_liabilities,capital_lease_obligations,long_term_debt,current_long_term_debt,long_term_debt_noncurrent,short_long_term_debt_total,other_current_liabilities,other_non_current_liabilities,total_shareholder_equity,treasury_stock,retained_earnings,common_stock,common_stock_shares_outstanding,symbol,sector,year,company_id,name,adjusted_date,cf_id,operating_cashflow,payments_for_operating_activities,proceeds_from_operating_activities,change_in_operating_liabilities,change_in_operating_assets,depreciation_depletion_and_amortization,capital_expenditures,change_in_receivables,change_in_inventory,profit_loss,cashflow_from_investment,cashflow_from_financing,proceeds_from_repayments_of_short_term_debt,payments_for_repurchase_of_common_stock,payments_for_repurchase_of_equity,payments_for_repurchase_of_preferred_stock,dividend_payout,dividend_payout_common_stock,dividend_payout_preferred_stock,proceeds_from_issuance_of_common_stock,proceeds_from_issuance_of_long_term_debt_and_capital_securities_net,proceeds_from_issuance_of_preferred_stock,proceeds_from_repurchase_of_equity,proceeds_from_sale_of_treasury_stock,change_in_cash_and_cash_equivalents,change_in_exchange_rate,net_income,income_id,gross_profit,total_revenue,cost_of_revenue,costof_goods_and_services_sold,operating_income,selling_general_and_administrative,research_and_development,operating_expenses,investment_income_net,net_interest_income,interest_income,interest_expense,non_interest_income,other_non_operating_income,depreciation,depreciation_and_amortization,income_before_tax,income_tax_expense,interest_and_debt_expense,net_income_from_continuing_operations,comprehensive_income_net_of_tax,ebit,ebitda,market_id,date,open,high,low,close,volume,shares,market_cap,gross_margin,net_profit_margin,roe,roa,current_ratio,debt_to_equity,debt_to_assets,eps,p/e_ratio,p/s_ratio,p/b_ratio,dividend_per_share,dividend_yield
0,0,2023-12-31,264811000000,23987000000,6215000000,6525000000,0,13813000000,272139000000,59686000000,58701000000,183035000000,123767000000,59268000000,9694000000,9385000000,310000000,3959000000,12333000000,182108000000,40198000000,12437000000,3860000000,2069000000,2069000000,147602000000,0,103200000000,2069000000,92200000000,103200000000,22450000000,20122000000,82703000000,7517000000,52892000000,48000000,4008000000,CMCSA,Comunicacion,2023,0,Comcast Corp,2023-12-29,0,28501000000,4674000000,0,-520000000,0,14336000000,15540000000,0,1516000000,15107000000,-7161000000,-19850000000,-660000000,11291000000,11291000000,0,4766000000,4766000000,0,0,6052000000,0,-11291000000,0,0,0,15388000000,0,107098000000,121572000000,14474000000,36762000000,23314000000,8992000000,0,15249000000,0,-4087000000,0,4087000000,0,592000000,8854000000,5482000000,20759000000,5371000000,4087000000,15107000000,16746000000,24846000000,30328000000,78761,2023-12-29,44.09,44.14,43.55,43.85,13695609,3817100000,167380000000,88.09,12.66,18.61,5.81,0.60,2.20,0.69,4.03,10.88,1.38,2.02,1.25,2.85
1,1,2022-12-31,257275000000,21826000000,4749000000,5151000000,0,12672000000,276545000000,55485000000,56900000000,193402000000,134908000000,58494000000,8142000000,7740000000,402000000,4406000000,12497000000,176332000000,27887000000,12544000000,3115000000,1743000000,1743000000,149019000000,0,100900000000,1743000000,86900000000,100900000000,11220000000,20395000000,80943000000,7517000000,51609000000,51000000,4277000000,CMCSA,Comunicacion,2022,0,Comcast Corp,2022-12-30,1,26413000000,

In [13]:
'''
# Calcular volatilidad diaria y ratios basados en volumen
merged_df['Daily Volatility'] = ((merged_df['high'] - merged_df['low']) / merged_df['close']) * 100
merged_df['Turnover Ratio'] = merged_df['volume'] / merged_df['shares']
'''

"\n# Calcular volatilidad diaria y ratios basados en volumen\nmerged_df['Daily Volatility'] = ((merged_df['high'] - merged_df['low']) / merged_df['close']) * 100\nmerged_df['Turnover Ratio'] = merged_df['volume'] / merged_df['shares']\n"

In [26]:
ratios_df

,ratios_id,symbol,name,company_id,sector,year,gross_margin,net_profit_margin,roe,roa,current_ratio,debt_to_equity,debt_to_assets,eps,p/e_ratio,p/s_ratio,p/b_ratio,dividend_per_share,dividend_yield,market_cap
0,0,CMCSA,Comcast Corp,0,Comunicacion,2023,88.09,12.66,18.61,5.81,0.60,2.20,0.69,4.03,10.88,1.38,2.02,1.25,2.85,167380000000
1,1,CMCSA,Comcast Corp,0,Comunicacion,2022,80.82,4.42,6.63,2.09,0.78,2.18,0.69,1.41,24.80,1.10,1.65,1.24,3.55,133484000000
2,2,CMCSA,Comcast Corp,0,Comunicacion,2021,80.60,12.17,14.73,5.13,0.85,1.87,0.65,3.71,13.57,1.65,2.00,1.19,2.36,192115000000
3,3,CMCSA,Comcast Corp,0,Comunicacion,2020,79.91,10.17,11.66,3.85,0.93,2.03,0.67,2.76,18.99,1.93,2.21,1.08,2.06,200016000000
4,4,CMCSA,Comcast Corp,0,Comunicacion,2019,80.03,11.99,15.78,4.96,0.84,2.18,0.69,3.42,13.15,1.58,2.07,0.98,2.18,171655000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,324,META,Meta Platforms Inc.,54,Tecnologia de la Informacion,2022,78.30,19.94,18.45,12.49,2.20,0.48,0.32,8.82,13.64,2.72,2.52,0.00,0.00,316374000000
325,325,META,Meta Platforms Inc.,54,Tecnologia de la Informacion,2021,80.77,33.43,31.53,23.72,3.15,0.33,0.25,14.98,22.45,7.51,7.08,0.00,0.00,884264000000
326,326,META,Meta Platforms Inc.,54,Tecnologia de la Informacion,2020,80.55,33.96,22.72,18.29,5.05,0.24,0.19,11.09,24.63,8.37,5.60,0.00,0.00,718138000000
327,327,META,Meta Platforms Inc.,54,Tecnologia de la Informacion,2019,81.91,26.19,18.29,13.86,4.40,0.32,0.24,7.03,29.20,7.65,5.34,0.89,0.43,539602000000
